# Functions

Topics:

1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

# How to declare a function

Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords.

In [1]:
function sayhi(name)
    println("Hi $name, it's great to see you!")
end

sayhi (generic function with 1 method)

In [2]:
function f(x)
    x^2
end

f (generic function with 1 method)

We can call either of these functions like this:

In [3]:
sayhi("C-3PO")

Hi C-3PO, it's great to see you!

In [4]:
f(42)

1764

Alternatively, we could have declared either of these functions in a single line

In [5]:
sayhi2(name) = println("Hi $name, it's great to see you!")

sayhi2 (generic function with 1 method)

Finally, we could have declared these as "anonymous" functions

In [6]:
sayhi3 = name -> println("Hi $name, it's great to see you!")

#3 (generic function with 1 method)

In [7]:
f3 = x -> x^2

#5 (generic function with 1 method)

In [8]:
sayhi3("Chewbaca")

Hi Chewbaca, it's great to see you!

In [9]:
f3(42)

1764

# Duck-typing in Julia

*"If it quacks like a duck, it's a duck."*

Julia functions will just work on whatever inputs make sense.

For example, `sayhi` works on the name of this minor tv character, written as an integer..

In [10]:
sayhi(55595472)

Hi 55595472, it's great to see you!


And `f` will work on a matrix.

In [11]:
A = rand(3,3)
A

3×3 Matrix{Float64}:
 0.215975   0.809662  0.711977
 0.522466   0.857456  0.291404
 0.0834421  0.466495  0.0999937

In [12]:
f(A)

3×3 Matrix{Float64}:
 0.529075  1.20125   0.460901
 0.585147  1.29419   0.650989
 0.270093  0.514205  0.205346

`f` will also work on a string like "hi" because * is defined for string inputs as string concatenation.

In [13]:
f("hi")

"hihi"

On the other hand, `f` will not work on a vector. Unlike `A^2` is well-defined, the meaning of `v^2` for a vector, `v`, is not a well-defined algebraic operation.

In [14]:
v = rand(3)

3-element Vector{Float64}:
 0.28439069509767156
 0.8651686443836704
 0.8067408613498601

In [15]:
f(v)  # This will not work

MethodError: MethodError: no method matching ^(::Vector{Float64}, ::Int64)
Closest candidates are:
  ^(!Matched::Union{AbstractChar, AbstractString}, ::Integer) at strings/basic.jl:718
  ^(!Matched::Complex{var"#s79"} where var"#s79"<:AbstractFloat, ::Integer) at complex.jl:818
  ^(!Matched::Complex{var"#s79"} where var"#s79"<:Integer, ::Integer) at complex.jl:820
  ...

# Mutating vs. non-mutating functions

By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.

In [16]:
v = [3,5,2]

3-element Vector{Int64}:
 3
 5
 2

In [17]:
sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [18]:
v

3-element Vector{Int64}:
 3
 5
 2

On the other hand, when we run `sort!(v)`, the contents `v` are sorted within array `v`.

In [19]:
sort!(v)

3-element Vector{Int64}:
 2
 3
 5

In [20]:
v

3-element Vector{Int64}:
 2
 3
 5

# Some higher order functions

## map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguements. `map` then applies that function to every element of the data structure you pass it. For example, executing

`   map(f, [1,2,3])`

will give you an output array where the function `f` has been applies to all elements of `[1,2,3]`

`   [f(1), f(2), f(3)] `

In [21]:
map(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Here we have squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as 

In [22]:
x -> x^3

#7 (generic function with 1 method)

In [23]:
map(x-> x^3, [1,2,3])

3-element Vector{Int64}:
  1
  8
 27

and now we have cubed all the elements of `[1, 2, 3]`

## Broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do everything `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`.

In [24]:
broadcast(f, [1,2,3])

3-element Vector{Int64}:
 1
 4
 9

and again, we have applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`. 

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

`
broadcast(f, [1,2,3])
`

is the same as

`
f.([1,2,3])
`

In [25]:
f.([1,2,3])

3-element Vector{Int64}:
 1
 4
 9

Notice again how different this is from calling

`f([1,2,3])`

We can square every element of a vector, but we can't square a vector!

To drive home the point, let's look at the difference between

`f(A)`

and 

`f.(A)`

for a matrix A.

In [26]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [27]:
f(A)

3×3 Matrix{Int64}:
  30   36   42
  66   81   96
 102  126  150

As before we see that for a matrix, A,

` f(A) = A^2 = A * A`

On the other hand,

In [28]:
B = f.(A)

3×3 Matrix{Int64}:
  1   4   9
 16  25  36
 49  64  81

contains the squares of all the entries of A.

This dot syntax for broadcasting allows us to write relatively complex compound element-wise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

In [29]:
A .+ 2 .* f.(A) ./ A

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

In [30]:
broadcast(x -> x+2*f(x)/x, A)

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

and the 2 will perform exactly the same.

# Exercises

**6.1**

Write a function `add_one` that adds 1 to its input.

In [32]:
function add_one(x)
    x + 1
end

add_one (generic function with 1 method)

In [33]:
@assert add_one(1) == 2

In [34]:
@assert add_one(11) == 12

**6.2** 

Use `map` or `broadcast` to increment every element of matrix A by ` and assign it to a variable A1.

In [38]:
A1 = map(add_one, A)

3×3 Matrix{Int64}:
 2  3   4
 5  6   7
 8  9  10

In [39]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

**6.3**

Use the broadcast dot syntax to increment every element of matrix A1 by 1 and store it in variable A2.

In [41]:
A2 = add_one.(A1)

3×3 Matrix{Int64}:
 3   4   5
 6   7   8
 9  10  11

In [42]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]